In [ ]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
%matplotlib inline

from entities import CoachingPracticeFinance, ExpenseLineItem, HoursLineItem

practice = CoachingPracticeFinance.load("../practice.json")

# Frame for the report

Let's sort out the bounds for the report, and set up a frame to merge into later.

In [ ]:
period_start = pd.to_datetime("2022-11-01")
period_end = pd.to_datetime("2023-10-31")
report_date_range = pd.bdate_range(period_start, period_end, freq="C", holidays=practice.statutory_holiday_list)
report_frame = pd.DataFrame(report_date_range, columns=["date"])
report_frame

# Prepare Budget Frame

Gather transaction agreement data into a frame bounded by the report range, and spread it out day by day.

In [ ]:
ta_array = []
for ta in practice.transaction_agreements:
    ta_range = pd.bdate_range(ta.start_date, ta.end_date, freq="C", holidays=practice.statutory_holiday_list)
    for date in ta_range:
        if date in report_date_range:
            ta_dict = ta.model_dump()
            ta_dict["rate"] = float(ta.rate.root)
            ta_dict["date"] = date
            ta_dict["hours"] = float(ta.hours/len(ta_range))
            ta_array.append(ta_dict)

ta_frame = pd.DataFrame(ta_array)
ta_frame["amount"] = ta_frame.rate * ta_frame.hours
ta_frame

In [ ]:
ta_frame.amount.sum()

In [ ]:
budget_summary = ta_frame[["date", "amount"]].rename(columns={"amount": "budget"})
budget_summary = budget_summary.groupby("date").sum()#.reset_index()
budget_summary

In [ ]:
budget_summary.plot()

In [ ]:
budget_summary.budget.sum()

# Merge the budget frame into the report frame

In [ ]:
# Merge the ta frame into the report frame
# Days not populated with ta data will be NaN
report_frame = report_frame.merge(budget_summary, on=["date"], how="outer")
report_frame

# Looking around while we're here

In [ ]:
temp = report_frame.copy()
temp["year"] = temp.date.dt.year
temp["month"] = temp.date.dt.month
temp = temp.groupby(["year", "month"]).aggregate({"budget": "sum"})
temp

In [ ]:
temp.budget.sum()

# Prepare Actuals Frame

This comes from invoice data, spread out day by day over the indicated periods in the hours line items.

In [ ]:
ta_array = []
for consultancy in practice.consultancies:
    for invoice in consultancy.invoices:
        for line_item in invoice.line_items:
            line_item_dict = line_item.model_dump()
            line_item_dict["amount"] = float(line_item.amount.root)
            if line_item_dict["tag"] == "Hours":
                hours_range = pd.bdate_range(line_item.period_start, line_item.period_end, freq="C", holidays=practice.statutory_holiday_list)
                days = len(hours_range)
                daily_hours = line_item_dict["hours"] / days
                daily_amount = line_item_dict["amount"] / days
                for date in hours_range:
                    dated_dict = line_item_dict.copy()
                    dated_dict["date"] = date
                    dated_dict["amount"] = daily_amount
                    dated_dict["hours"] = daily_hours
                    ta_array.append(dated_dict)
            else:
                line_item_dict["date"] = pd.Timestamp(invoice.issue_date)
                ta_array.append(line_item_dict)
# rows
actuals_frame = pd.DataFrame(ta_array)
actuals_frame

In [ ]:
actuals_summary = actuals_frame[["date", "amount"]].rename(columns={"amount": "actual"})
actuals_summary = actuals_summary.groupby("date").sum()#.reset_index()
actuals_summary

In [ ]:
report_frame = report_frame.merge(actuals_summary, on="date", how='outer')
report_frame

In [ ]:
report_frame['year'] = report_frame['date'].dt.year
report_frame['month'] = report_frame['date'].dt.month
report_frame['budget'] = report_frame['budget'].fillna(0)
report_frame['actual'] = report_frame['actual'].fillna(0)
report_frame.groupby(["year", "month"]).aggregate({"date": "first", "budget": "sum", "actual": "sum"}).plot(y=["budget", "actual"])

In [ ]:
report_frame.budget.sum()

In [ ]:
report_frame.actual.sum()

In [ ]:
report_frame